<br>

### Exercício: Regressão Linear:
__Parte 1:__

1- Usando a função getData(), carregue os dados disponibilizados.

2- Separe parte dos dados para o dataset de teste.

3- Usando a metodologia de validação cruzada, teste diferentes parâmetros da regLinear - diferentes learning_rates e num_steps - para escolher a melhor combinação de parâmetros.

4- Implemente a regressão linear do scikit-learn e compare os resultados obtidos.

__Parte 2 (Introdução):__

Para cada variável explicativa $X_1, .., X_5$, crie outras variáveis usando o __quadrado__ de cada um delas. Desta forma, o conjunto final será de 10 variáveis, em que:

$X_6 = (X_1)^{2}$, $X_7 = (X_2)^{2}$, $X_8 = (X_3)^{2}$, $X_9 = (X_4)^{2}$, $X_{10} = (X_5)^{2}$.

Ao treinarmos uma regressão linear com essas 10 variáveis, a predição é da forma:

$y_{pred} = \theta_0 + \theta_1 \cdot X_1 + .. + \theta_5 \cdot X_5 + \theta_6 \cdot (X_1)^{2} + .. + \theta_{10} \cdot (X_5)^{2}$

Como estamos usando o quadrado das variáveis explicativas, dizemos que temos um __modelo de regressão polinomial de grau 2__. Podemos ter variações deste modelo:

-Podemos aumentar o grau: basta mudar a potência que elevamos as variáveis. Por exemplo, podemos incluir o __cubo__ das variáveis e termos um modelo polinomial de ordem 3.

-Podemos ter __interações__ entre as variáveis: multiplicações entre as variáveis.

Exemplo:

$y_{pred} = \theta_0 + \theta_1 \cdot X_1 + .. + \theta_5 \cdot X_5 + \theta_6 \cdot (X_1)^{2} + .. + \theta_{10} \cdot (X_5)^{2} + \theta_{11} \cdot (X_1)^{3} + \theta_{12} \cdot V1 + \theta_{13} \cdot V2$,

onde

$V_1 = X_1 \cdot X_2$ e $V_2 = (X_2)^{2} \cdot X_4$

__Parte 2 (Exercício):__

1- Estude o link:
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

em que é discutido como criar modelos polinomiais com o scikit-learn de forma detalhada.

2- Repita os passos da primeira parte, mas agora considerando polinômios de graus 2 ou mais.

3- Inclua regularização Ridge e Lasso nas análises e teste os resultados para diferentes parâmetros $\alpha$.

<br>

### Exercício: Regressão Logística:

__Parte 1:__

Crie uma classe regLogistica para treinar o modelo de regressão logística. Essa classe deve ser usada para problemas de classificação binária, cuja variável target assume os valores: 0 (classe negativa) e 1 (classe positiva).

O método construtor dessa classe deve possuir 3 parâmetros: learning_rate, num_steps e limiar.

Os outros médotos devem ser:

    - médoto fit: para treinar o modelo - usando gradient descent
    
    - médoto predict_proba: para retornar a probabilidade da classe 1
    
    - médoto predict: retornar a classe predita: 0 ou 1 - dependente do limiar
    
__Parte 2:__

Usando a função getData2(), carregue o dataset disponibilizado.

Use a regLogistica, classe criada na parte 1 do exercício, para treinar modelos nestes dados. Use validação cruzada para seleção dos parâmetros. Considere diferentes métricas de classificação e justifique as escolhas.

**Exercicio 1 - Parte 1**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_friedman1, make_classification

In [ ]:
#função para acessar os dados do exercício 1

def getData():
    X, y = make_friedman1(n_samples = 10000, n_features = 5, noise = 5.0, random_state = 0)
    return X, y

In [ ]:
X, y = getData()

In [ ]:
Xtreino, Xteste, ytreino, yteste = X[:7500], X[7500:], y[:7500], y[7500:]

In [ ]:
Xtreino.shape, Xteste.shape, ytreino.shape, yteste.shape

In [ ]:
#classe regLinear para exercício

class regLinear():
    
    def __init__(self, learning_rate, num_steps):
        self.learning_rate = learning_rate
        self.num_steps = num_steps
        
    def fit(self, X, y):
        y = y.reshape(-1,1)
        m = X.shape[0] 
        k = X.shape[1] 
        theta = np.random.randn(k+1,1) 
        X_b = np.c_[np.ones((m, 1)), X] 
        for step in range(self.num_steps):
            gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
            theta = theta - self.learning_rate * gradients
        self.theta_final = theta
        print("modelo treinado.")
        
    def predict(self, X):
        m = X.shape[0]
        X_b = np.c_[np.ones((m, 1)), X]
        preds = X_b.dot(self.theta_final)
        return preds.reshape(-1,)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
from sklearn.metrics import mean_squared_error

def MSE(ytrue, ypred):
    return mean_squared_error(y_true = ytrue, y_pred = ypred)

In [ ]:
def validacao_cruzada(classificador, 
                      X, 
                      y, 
                      metrica, 
                      num_folds, 
                      print_info = False, 
                      nome_metrica = None):
    
    lista_metrica_treino = []
    lista_metrica_validacao = []
    
    kf = KFold(n_splits = num_folds)
    for train_index, val_index in kf.split(X, y):
        
        Xtrain_folds = X[train_index]
        ytrain_folds = y[train_index]
        Xval_fold = X[val_index]
        yval_fold = y[val_index]
        
        classificador.fit(X = Xtrain_folds, y = ytrain_folds)
        
        pred_treino = classificador.predict(Xtrain_folds)
        pred_validacao = classificador.predict(Xval_fold)
        
        lista_metrica_treino.append(metrica(ytrue = ytrain_folds, ypred = pred_treino))
        lista_metrica_validacao.append(metrica(ytrue = yval_fold, ypred = pred_validacao))
        
    if print_info:
        print("Métrica: " + nome_metrica)
        print('média treino:', np.mean(lista_metrica_treino))
        print('média validação:', np.mean(lista_metrica_validacao))
        
    return lista_metrica_treino, lista_metrica_validacao

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.075, num_steps = 200),
                 X = Xtreino,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.1, num_steps = 400),
                 X = Xtreino,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 1, num_steps = 50),
                 X = Xtreino,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.075, num_steps = 100),
                 X = Xtreino,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
rL = regLinear(learning_rate = 0.1, num_steps = 400)

In [ ]:
rL.fit(Xtreino, ytreino)

In [ ]:
pred = rL.predict(Xteste)

In [ ]:
resultado = MSE(ytrue = yteste, ypred = pred)
resultado

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
validacao_cruzada(classificador = LinearRegression(),
                 X = Xtreino,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
lm = LinearRegression()
lm.fit(Xtreino, ytreino)

In [ ]:
pred1 = lm.predict(Xteste)

In [ ]:
resultado1 = MSE(ytrue = yteste, ypred = pred1)
resultado1

O modelo onde tivemos o melhor resultado foi o com uma "learning rate = 0.1" e "num steps = 400", seu resultado na base de teste foi bem semelhante ao resultado utilizando a regressão linear do scikit-learn.

**Exercicio 1 - Parte 2**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(2)
Xtreino_P2 = poly.fit_transform(Xtreino)

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.075, num_steps = 200),
                 X = Xtreino_P2,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.1, num_steps = 400),
                 X = Xtreino_P2,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 1, num_steps = 50),
                 X = Xtreino_P2,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.075, num_steps = 100),
                 X = Xtreino_P2,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
poly = PolynomialFeatures(20)
Xtreino_P20 = poly.fit_transform(Xtreino)

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.075, num_steps = 200),
                 X = Xtreino_P20,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.1, num_steps = 400),
                 X = Xtreino_P20,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 1, num_steps = 50),
                 X = Xtreino_P20,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.075, num_steps = 100),
                 X = Xtreino_P20,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
poly = PolynomialFeatures(6)
Xtreino_P6 = poly.fit_transform(Xtreino)

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.075, num_steps = 200),
                 X = Xtreino_P6,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.1, num_steps = 400),
                 X = Xtreino_P6,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 1, num_steps = 50),
                 X = Xtreino_P6,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = regLinear(learning_rate = 0.075, num_steps = 100),
                 X = Xtreino_P6,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
validacao_cruzada(classificador = LinearRegression(),
                 X = Xtreino_P6,
                 y = ytreino,
                 metrica = MSE,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'MSE')

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [ ]:
def polyFitReg(X, y, grau, base_model, base_model_name):
    
    polybig_features = PolynomialFeatures(degree = grau, include_bias = False)
    std_scaler = StandardScaler()
    basemodel = base_model
    
    #criando um pipeline: sequencia de execução de passos
    polynomial_regression = Pipeline([
            ("poly_features", polybig_features),
            ("std_scaler", std_scaler),
            (base_model_name, base_model),
        ])
    
    polynomial_regression.fit(X, y)
    return polynomial_regression

In [ ]:
grau = 6 

for alpha in [0, 0.001, 0.01, 0.1, 1, 10, 100]:
    
    model_name = 'Ridge_ alpha: '+str(alpha)
    polyfit = polyFitReg(X = Xtreino, 
                         y = ytreino, 
                         grau = grau, 
                         base_model = Ridge(alpha = alpha), 
                         base_model_name = model_name)
    
    ypoly_novo = polyfit.predict(Xteste)

    print(model_name)
    plt.figure(figsize=[6,5])
    plt.plot(Xteste, ypoly_novo, label=str(grau), lw=2, c = 'red')
    plt.scatter(Xteste[:,0], yteste, c = "blue")
    plt.axis([-3, 3, 0, 10])
    plt.show()
    print("MSE:")
    print('treino:', np.mean(np.square(y - polyfit.predict(X))))
    print('teste:', np.mean(np.square(yteste - polyfit.predict(Xteste))))
    print("------------------------------------------------\n\n")

In [ ]:
grau = 6 

for alpha in [0, 0.001, 0.01, 0.1, 1, 10, 100]:
    
    model_name = 'Lasso_ alpha: '+str(alpha)
    polyfit = polyFitReg(X = Xtreino, 
                         y = ytreino, 
                         grau = grau, 
                         base_model = Lasso(alpha = alpha), 
                         base_model_name = model_name)
    
    ypoly_novo = polyfit.predict(Xteste)

    print(model_name)
    plt.figure(figsize=[6,5])
    plt.plot(Xteste, ypoly_novo, label=str(grau), lw=2, c = 'red')
    plt.scatter(Xteste[:,0], yteste, c = "blue")
    plt.axis([-3, 3, 0, 10])
    plt.show()
    print("MSE:")
    print('treino:', np.mean(np.square(y - polyfit.predict(X))))
    print('teste:', np.mean(np.square(yteste - polyfit.predict(Xteste))))
    print("------------------------------------------------\n\n")

Após todos os modelos testados, onde tivemos o melhor resultado foi utilizando 6 graus de polinômios e utilizando a regularização de Lasso com alpha em 0.01.

**Exercicio 2 - Parte 1**

In [ ]:
def logLossCost(ytrue, ypred_probs):
    return (ytrue * np.log(ypred_probs) + (1 - ytrue) * np.log(1 - ypred_probs)).mean() * -1

In [ ]:
def sigmoid(t):
    return 1 / (1 + np.exp(-t))

In [ ]:
class regLogistica():
    
    def __init__(self, learning_rate, num_steps, limiar):
        self.learning_rate = learning_rate
        self.num_steps = num_steps
        self.linear = linear
        
    def fit(self, X, y):
        X_b = np.c_[np.ones(X.shape[0]), X]
        theta = np.random.randn(X_b.shape[1],1)
        for step in range(self.num_steps):
            yscores = sigmoid(X_b.dot(theta))
            gradient = X_b.T.dot(yscores - y)
            theta = theta - self.learning_rate * gradient
        self.theta_final = theta
        print("modelo treinado.")
        
    def predict_proba(self, X):
        X_b = np.c_[np.ones(X.shape[0]), X]
        probs = sigmoid(X_b.dot(self.theta_final))
        return probs
        
    def predict(self, probs):
        linear = self.linear
        ypred = np.where(probs > limiar, 1, 0)
        return ypred.reshape(-1,)

**Exercicio 2 - Parte 2**

In [ ]:
#função para acessar os dados do exercício 2

def getData2():
    X, y = make_classification(n_classes=2, n_features=5, n_samples=10000, random_state = 0)
    return X, y

In [ ]:
X, y = getData2()

In [ ]:
Xtreino, Xteste, ytreino, yteste = X[:7500], X[7500:], y[:7500], y[7500:]

In [ ]:
ytreino = ytreino.reshape(-1,1)

yteste = yteste.reshape(-1,1)

In [ ]:
Xtreino.shape, Xteste.shape, ytreino.shape, yteste.shape

In [ ]:
def validacao_cruzada_log(classificador, 
                      X, 
                      y, 
                      metrica, 
                      num_folds, 
                      print_info = False, 
                      nome_metrica = None):
    
    lista_metrica_treino = []
    lista_metrica_validacao = []
    
    kf = KFold(n_splits = num_folds)
    for train_index, val_index in kf.split(X, y):
        
        Xtrain_folds = X[train_index]
        ytrain_folds = y[train_index]
        Xval_fold = X[val_index]
        yval_fold = y[val_index]
        
        classificador.fit(X = Xtrain_folds, y = ytrain_folds)
        
        probs_treino = classificador.predict_proba(Xtrain_folds)
        probs_validacao = classificador.predict_proba(Xval_fold)
        
        pred_treino = classificador.predict(probs_treino)
        pred_validacao = classificador.predict(probs_validacao)
        
        lista_metrica_treino.append(metrica(y_true = ytrain_folds, y_pred = pred_treino))
        lista_metrica_validacao.append(metrica(y_true = yval_fold, y_pred = pred_validacao))
        
    if print_info:
        print("Métrica: " + nome_metrica)
        print('média treino:', (lista_metrica_treino[0]+lista_metrica_treino[1]+lista_metrica_treino[2]+lista_metrica_treino[3]+lista_metrica_treino[4])/5)
        print('média validação:', (lista_metrica_validacao[0]+lista_metrica_validacao[1]+lista_metrica_validacao[2]+lista_metrica_validacao[3]+lista_metrica_validacao[4])/5)
        
    return lista_metrica_treino, lista_metrica_validacao

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
validacao_cruzada_log(classificador = regLogistica(learning_rate = 0.075, num_steps = 200, limiar = 0.5),
                 X = Xtreino,
                 y = ytreino,
                 metrica = confusion_matrix,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'Matrix de Confusão')

In [ ]:
validacao_cruzada_log(classificador = regLogistica(learning_rate = 0.1, num_steps = 400, limiar = 0.5),
                 X = Xtreino,
                 y = ytreino,
                 metrica = confusion_matrix,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'Matrix de Confusão')

In [ ]:
validacao_cruzada_log(classificador = regLogistica(learning_rate = 0.075, num_steps = 400, limiar = 0.5),
                 X = Xtreino,
                 y = ytreino,
                 metrica = confusion_matrix,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'Matrix de Confusão')

In [ ]:
validacao_cruzada_log(classificador = regLogistica(learning_rate = 0.075, num_steps = 200, limiar = 0.3),
                 X = Xtreino,
                 y = ytreino,
                 metrica = confusion_matrix,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'Matrix de Confusão')

In [ ]:
validacao_cruzada_log(classificador = regLogistica(learning_rate = 0.1, num_steps = 400, limiar = 0.3),
                 X = Xtreino,
                 y = ytreino,
                 metrica = confusion_matrix,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'Matrix de Confusão')

In [ ]:
validacao_cruzada_log(classificador = regLogistica(learning_rate = 0.075, num_steps = 400, limiar = 0.3),
                 X = Xtreino,
                 y = ytreino,
                 metrica = confusion_matrix,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'Matrix de Confusão')

In [ ]:
validacao_cruzada_log(classificador = regLogistica(learning_rate = 0.075, num_steps = 200, limiar = 0.7),
                 X = Xtreino,
                 y = ytreino,
                 metrica = confusion_matrix,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'Matrix de Confusão')

In [ ]:
validacao_cruzada_log(classificador = regLogistica(learning_rate = 0.1, num_steps = 400, limiar = 0.7),
                 X = Xtreino,
                 y = ytreino,
                 metrica = confusion_matrix,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'Matrix de Confusão')

In [ ]:
validacao_cruzada_log(classificador = regLogistica(learning_rate = 0.075, num_steps = 400, limiar = 0.7),
                 X = Xtreino,
                 y = ytreino,
                 metrica = confusion_matrix,
                 num_folds = 5,
                 print_info = True,
                 nome_metrica = 'Matrix de Confusão')

In [ ]:
log = regLogistica(learning_rate = 0.075, num_steps = 200, limiar = 0.3)

In [ ]:
log.fit(X = Xtreino, y = ytreino)

In [ ]:
probs = log.predict_proba(X = Xteste)
probs

In [ ]:
ypred = log.predict(probs)
ypred

In [ ]:
confusion_matrix(y_true = yteste, y_pred = ypred)

Entre os treinamentos realizados com as diferentes metricas de classificação, a que se saiu melhor na média da matriz de confusão foi com uma learning rate de **0,075**, **200** steps e um limiar de **0.3**. Quando aplicado nas bases de teste o resultado se manteve satisfatório.